# Euler - Maruyama for Heston Stochastic Volatility Model 

In [12]:
import pandas as pd
import numpy as np
from scipy.stats import norm



kappa = 1              #mean-reversion speed
theta = 0.09           #long-term average volatility
eta   = 1              #volatility of vol process
rho   = -0.3           #correlation between stock and vol
v0    = 0.09           #initial volatility
r     = 0              #risk-free interest rate
tau   = 1              #time to maturity
S0    = 100            #initial share price
K     = [80,100,120]   #strike price



def HestonCallMonteCarlo(kappa, theta, eta, rho, v0, r, tau, S0, K, nSteps=1000, nPaths=3000):
    
    n = nSteps
    N = nPaths
    
    dt = tau / n
    alpha = norm.ppf(1-.05/2)
    negCount = 0
    
    S = np.repeat(S0,N)
    v = np.repeat(v0,N)
    
    for i in range(1,n):
        W1 = np.random.normal(0,1,N)
        W2 = np.random.normal(0,1,N)
        W2 = rho*W1 + np.sqrt(1 - rho**2)*W2
      
        sqvdt = np.sqrt(v*dt)
        S = S*np.exp((r-v/2)*dt + np.sqrt(v * dt) * W1)
    
        v = v + kappa*(theta - v)* dt + eta * sqvdt * W2
        negCount = negCount + len(v[v < 0])
        v[v < 0] = 0
     
      
    
    negCount = negCount / (n*N)
    
    ## Evaluate the European call value 
    V  =  np.exp(-r*tau)*np.maximum(S-K,0)  
    AV =  np.mean(V) # mean of call price
    AVdev = alpha * np.std(V) / np.sqrt(N) # standard error 
    rmse  = np.sqrt(np.var(V)/N)
    return(AV,AV-AVdev,AV+AVdev,rmse)


h1 = HestonCallMonteCarlo(kappa, theta, eta, rho, v0, r, tau, S0, K=K[0], nSteps=1000, nPaths=5000)
h2 = HestonCallMonteCarlo(kappa, theta, eta, rho, v0, r, tau, S0, K=K[1], nSteps=1000, nPaths=5000)
h3 = HestonCallMonteCarlo(kappa, theta, eta, rho, v0, r, tau, S0, K=K[2], nSteps=1000, nPaths=5000)
df = pd.DataFrame({'Heston K = 80' :h1,
                   'Heston K = 100':h2,
                   'Heston K = 120':h3 })
df.index = ['value', 'Lower 95%', 'Upper 95%', 'RMSE']
df

,Heston K = 80,Heston K = 100,Heston K = 120
value,24.267887,10.079237,3.386596
Lower 95%,23.561164,9.571718,3.024277
Upper 95%,24.974611,10.586756,3.748916
RMSE,0.360580,0.258943,0.184860
